## Vehicle Re-Identification Training Example

In [8]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import torchreid
import sys
import os.path as osp
import pandas as pd
from torchreid.data import ImageDataset

In [9]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.makedirs("./log", exist_ok=True)


In [10]:
# Some of the available models, which you can directly use with torchreid module
models_list = ['osnet_x1_0' ,'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'resnet50','resnet152']

In [11]:
class VeRiDataset(ImageDataset):

    # All you need to do here is to generate three lists,
    # which are train, query and gallery.
    # Each list contains tuples of (img_path, pid, camid),
    # where
    # - img_path (str): absolute path to an image.
    # - pid (int): person ID, e.g. 0, 1.
    # - camid (int): camera ID, e.g. 0, 1.
    # Note that
    # - pid and camid should be 0-based.
    # - query and gallery should share the same pid scope (e.g.
    #   pid=0 in query refers to the same person as pid=0 in gallery).  --> Very important!
    # - train, query and gallery share the same camid scope (e.g.
    #   camid=0 in train refers to the same camera as camid=0
    #   in query/gallery).

    def __init__(self, root='', **kwargs):
        PATH_TO_DATASET = "/fast/ashutosh-sekilab/vehicle_reid/veri/" # This is the folder path where you have extracted the zip file and it contains the gallery, query and train images
        FOLDER_PATH, DATASET_TYPES, IMAGES = next(os.walk(f"{PATH_TO_DATASET}")) 
        training_array = []
        testing_array = []
        query_array = []

        for dataset_type in DATASET_TYPES:
            _, _, IMAGES = next(os.walk(f"{FOLDER_PATH}/{dataset_type}/"))

            if dataset_type == "image_train" or dataset_type == "image_gallery":

                labelSet = []
                for idx, image in enumerate(IMAGES):
                    pid = int(image.split('.jpg')[0].split('_')[-1])
                    labelSet.append(pid)
                label_dict = {label: index for index, label in enumerate(set(labelSet))}
                
                for image in IMAGES:
                    cam_id = int(image.split('.jpg')[0].split('_')[-2])
                    car_id = label_dict[int(image.split('.jpg')[0].split('_')[-1])]

                    if dataset_type == "image_gallery":
                        testing_array.append((FOLDER_PATH +  dataset_type + "/" + image, car_id, cam_id))
                    else:
                        training_array.append((FOLDER_PATH +  dataset_type + "/" + image, car_id, cam_id))
                        
            elif dataset_type == "image_query":
                for image in IMAGES:
                    cam_id = int(image.split('.jpg')[0].split('_')[-2])
                    car_id = label_dict[int(image.split('.jpg')[0].split('_')[-1])]
                    query_array.append((FOLDER_PATH +  dataset_type + "/" + image, car_id, cam_id))

            
        train = training_array
        query = query_array
        gallery = testing_array

        super(VeRiDataset, self).__init__(train, query, gallery, **kwargs)

In [12]:
try:
    torchreid.data.register_image_dataset('synthtetic_veri_reid', VeRiDataset)
except Exception as e:
    print(e)

In [13]:
# You can change these hyperparameters as you wish
datamanager = torchreid.data.ImageDataManager(
    root='',
    sources='synthtetic_veri_reid',
    height = 256,
    width = 256,
    transforms =['random_flip', 'random_crop'])

Building train transforms ...
+ resize to 256x256
+ random flip
+ random crop (enlarge to 288x288 and crop 256x256)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x256
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded VeRiDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   600 |    50949 |        85
  query    |    50 |      424 |        85
  gallery  |    50 |     3823 |        85
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded VeRiDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   600 |    50949 |        85
  query    |    50 |      424 |        85
  g

In [14]:
LOSS = "SOFTMAX" # Define softmax loss, else triple loss will be usd

In [15]:
for current_model in models_list:
    print(current_model)
    if LOSS == "SOFTMAX":
        model = torchreid.models.build_model(
            name=f'{current_model}',
            num_classes=datamanager.num_train_pids,
            loss='softmax'
        )
    else:
        model = torchreid.models.build_model(
            name=f'{current_model}',
            num_classes=datamanager.num_train_pids,
            loss='triplet'
        )

    model = model.cuda()
    optimizer = torchreid.optim.build_optimizer(
        model,
        optim='adam',
        lr=0.0003
    )

    scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=20
    )

    if LOSS == "SOFTMAX":
        engine = torchreid.engine.ImageSoftmaxEngine(
            datamanager,
            model,
            optimizer=optimizer,
            scheduler=scheduler,
            
        )
    else:
        print("Using ImageTripletEngine")
        engine = torchreid.engine.ImageTripletEngine(
            datamanager,
            model,
            optimizer=optimizer,
            scheduler=scheduler,
        )

    original_stdout = sys.stdout
    with open(f"log/log_{current_model}.txt", 'a') as f:
        sys.stdout = f
        engine.run(
        save_dir=f"log/{current_model}",
        max_epoch=60,
        eval_freq=5,
        print_freq=10,
        test_only=False
        )
        sys.stdout = original_stdout

    del engine
    del model
    del optimizer
    del scheduler

osnet_x1_0
Successfully loaded imagenet pretrained weights from "/home/mdxuser/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
